<a href="https://colab.research.google.com/github/JoaoOliveira6/2023-Q3-PLN-ATIVIDADE-PRATICA-04/blob/main/2023_Q3_PLN_ATIVIDADE_PR%C3%81TICA_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2023.Q3]**
Prof. Alexandre Donizeti Alves

### **ATIVIDADE PRÁTICA 04 [Uso da API da OpenAI com técnicas de PLN]**


A **ATIVIDADE PRÁTICA 04** deve ser feita utilizando o **Google Colab** com uma conta sua vinculada ao Gmail. O link do seu notebook, armazenado no Google Drive, além do link de um repositório no GitHub e os principais resultados da atividade, devem ser enviados usando o seguinte formulário:

> https://forms.gle/GzwCq3R7ExtE9g9a8


**IMPORTANTE**: A submissão deve ser feita até o dia **26/11 (domingo)** APENAS POR UM INTEGRANTE DA EQUIPE, até às 23h59. Por favor, lembre-se de dar permissão de ACESSO IRRESTRITO para o professor da disciplina de PLN.

### **EQUIPE**

---

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01:**

João Pedro Balceiro Rodrigues - RA: 11201811336

**Integrante 02:**

João Vitor Oliveira dos Santos - RA: 11201811164

### **LIVRO**
---

`Processamento de Linguagem Natural - Conceitos, Técnicas e Aplicações em Português.`

>

Disponível gratuitamente em:
  
  > https://brasileiraspln.com/livro-pln/1a-edicao/.


**POR FAVOR, PREENCHER OS CAPITULOS SELECIONADOS PARA A SUA EQUIPE:**

`Primeiro capítulo: ` 5

`Segundo capítulo:` 21



### **DESCRIÇÃO**
---

Implementar um `notebook` no `Google Colab` que faça uso da **API da OpenAI** aplicando, no mínimo, 3 técnicas de PLN. As técnicas devem ser aplicadas nos 2 (DOIS) capítulos do livro **Processamento de Linguagem Natural - Conceitos, Técnicas e Aplicações em Português**.

>

**RESTRIÇÃO**: É obrigatório usar o *endpoint* "*`Chat Completions`*".

>

As seguintes técnicas de PLN podem ser usadas:

*   Correção Gramatical
*   Classificação de Textos
*   Análise de Sentimentos
*   Detecção de Emoções
*   Extração de Palavras-chave
*   Tradução de Textos
*   Sumarização de Textos
*   **Similaridade de Textos**
*   **Reconhecimento de Entidades Nomeadas**
*   **Sistemas de Perguntas e Respostas**

>

Os capítulos devem ser os mesmos selecionados na **ATIVIDADE PRÁTICA 02**. Para consultar os capítulos, considere a seguinte planilha:

>

> https://docs.google.com/spreadsheets/d/1ZutzQ3v1OJgsgzCvCwxXlRIQ3ChXNlHNvB63JQvYsbo/edit?usp=sharing

>
>

**IMPORTANTE:** É obrigatório usar o e-mail da UFABC. Não é permitido alterar os capítulos já selecionados.



### **CRITÉRIOS DE AVALIAÇÃO**
---


Serão considerados como critérios de avaliação as técnicas usadas e a criatividade envolvida na aplicação das mesmas.




###**IMPLEMENTAÇÃO**
___

####**IMPORTANDO BIBLIOTECAS**



In [61]:
print(f"Instalando a biblioteca da API da OpenAI")

!pip install openai

print("API da OpenAI instalada")

import requests
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import base64
import openai
from google.colab import files
from openai import OpenAI



Instalando a biblioteca da API da OpenAI
API da OpenAI instalada


#### **URL DOS CAPÍTULOS DO LIVRO**
---

In [60]:
urls = ['https://brasileiraspln.com/livro-pln/1a-edicao/parte3/cap5/cap5.html',
        'https://brasileiraspln.com/livro-pln/1a-edicao/parte9/cap21/cap21.html']

#### **CONFIGURAÇÃO DA CHAVE DE ACESSO DA API**
---


In [59]:
upload_arquivo = files.upload()

nome_arquivo = list(upload_arquivo.keys())[0]

#envio de txt com a chave token
with open(nome_arquivo, 'r') as file:
    chave_api = file.read()

openai.api_key = chave_api
client = OpenAI(
    api_key=chave_api,
)



Saving chave_api_pln.txt to chave_api_pln (5).txt


#### **TESTE DE CHAVE API**
---

In [62]:
client = OpenAI(
    api_key=chave_api,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "O Brasil é um país",
        }
    ],
    model="gpt-3.5-turbo"
)

print(chat_completion.choices[0].message.content)

localizado na América do Sul, sendo o quinto maior país em extensão territorial. É conhecido por sua grande diversidade cultural, com influências indígenas, africanas e europeias. Sua capital é Brasília, e as maiores cidades são São Paulo, Rio de Janeiro e Salvador. 

O Brasil possui uma economia diversificada, com destaque para a produção agrícola, mineração, indústria e serviços. É considerado um país em desenvolvimento, tendo enfrentado desafios socioeconômicos ao longo de sua história.

Apesar das riquezas naturais e culturais, o Brasil também enfrenta problemas sociais, como desigualdade de renda, falta de acesso a serviços básicos e altos índices de violência.

O país possui uma democracia representativa, com um presidente eleito e um sistema de governo federal. A língua oficial é o português e a religião predominante é o cristianismo, principalmente a Igreja Católica.

O Brasil é conhecido internacionalmente por suas festas populares, como o carnaval, e pelas suas belezas natura

####**EXTRAÇÃO DE PALAVRAS-CHAVE POR CAPITULOS**
---

In [58]:
endpoint = "https://api.openai.com/v1/chat/completions"

def retornaParalavrasChaveEmArray(texto=0):
    if texto == 0:
      texto = ""

    mensagem_prompt = "extraia as palavras-chave do seguinte texto, retornando-as no padrão a seguir: \npadrão: \npalavra1|palavra2|palavra3|palavra4"
    mensagem_texto = "texto: \n" + texto

    parameters = {
        "model": "gpt-3.5-turbo-0613",
        "max_tokens":1000,
        "temperature":0,
        "messages": [
            {"role": "system", "content": mensagem_prompt},
            {"role": "user", "content": mensagem_texto}
        ]
    }

    headers = {
        "Content-Type":"application/json",
        "Authorization": f"Bearer {chave_api}"
    }

    resposta = requests.post(endpoint, json=parameters, headers=headers)
    retorno_chat = resposta.json()

    novo_texto = retorno_chat.get('choices', [{}])[0].get('message', {}).get('content', 'error')
    if(novo_texto == 'error'):
        print(retorno_chat)

    return novo_texto

print("Extração de palavras-chave de Capítulos\n")
for url in urls:

    # Fazer uma solicitação HTTP para obter o conteúdo da página
    response = requests.get(url)

    # Verificar se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Extrair o conteúdo HTML da página
        html = response.text

        # Criar um objeto BeautifulSoup para analisar o HTML
        soup = BeautifulSoup(html, 'html.parser')

        # Encontrar todos os parágrafos (geralmente estão dentro de elementos <p>)
        all_paragraphs = soup.find_all('p')

        # Encontrar o capítulo do livro (geralmente estão dentro de elementos <h1>)
        chapter_book = soup.find_all('h1')
        chapter_book = chapter_book[0].get_text().strip()
        print(f"capítulo: {chapter_book}\n")

        # Verificar cada parágrafo
        textoCompleto = ""
        for i, paragraph in enumerate(all_paragraphs, start=1):
            if len(paragraph.get_text().split()) > 10:
              #vai formando a string de texto ao concatenar os paragrafos
              #medida tomada pois chamar a API paragrafo por paragrafo foi enviavel
              textoCompleto = textoCompleto + paragraph.get_text() + "\n"

        arrayPalavrasChave = retornaParalavrasChaveEmArray(textoCompleto)
        print(arrayPalavrasChave)
        print("-------------------------------------------------------------\n")

    else:
        print("Não foi possível acessar a página:", response.status_code)


Extração de palavras-chave de Capítulos

capítulo: 5  ExpressÃµes multipalavras

expressões multipalavras|sintaxe|semântica|operações sintáticas|definições|sentido|todo|engolir o sapo|conceitos fundamentais|navegar|tratamento computacional|palavras|lexemas|MWEs|sequências|contínuas|descontínuas|palavras componentes|lexemas|idiossincrasia|lexical|morfológica|sintática|semântica|idiossincrasias|coesão sintática|elementos lexicalizados|compostos|colocações|metáforas|classificação|núcleo|verbo|diretrizes de anotação|categorização|ambiguidade|variabilidade|arbitrariedade|descoberta/detecção|identificação|recursos|português brasileiro|métricas|testes|avaliação|desempenho|pesquisas|desafios|modelos de língua|conhecimento linguístico|exemplos|fenômenos linguísticos|divertir|próximo capítulo
-------------------------------------------------------------

capítulo: 21  PLN na SaÃºde

{'error': {'message': 'Rate limit reached for gpt-3.5-turbo-0613 in organization org-EISejVRcM1gc4J9EOJ6rwD4y on r

####**TRADUÇÃO DE TEXTO PARA A LINGUA ESCOLHIDA**

In [66]:
def traducao(texto, idioma):
    mensagem_prompt = f"traduza o texto abaixo para o idioma {idioma}"
    mensagem_texto = "texto: \n" + texto

    parameters = {
        "model": "gpt-3.5-turbo-0613",
        "max_tokens":1000,
        "temperature":0,
        "messages": [
            {"role": "system", "content": mensagem_prompt},
            {"role": "user", "content": mensagem_texto}
        ]
    }

    headers = {
        "Content-Type":"application/json",
        "Authorization": f"Bearer {chave_api}"
    }

    resposta = requests.post(endpoint, json=parameters, headers=headers)
    retorno_chat = resposta.json()

    novo_texto = retorno_chat.get('choices', [{}])[0].get('message', {}).get('content', 'error')
    if(novo_texto == 'error'):
        print(retorno_chat)

    return novo_texto

print("Tradução de Capítulos\n")
idioma = input("Digite para qual idioma deseja traduzir: \n")
for url in urls:

    # Fazer uma solicitação HTTP para obter o conteúdo da página
    response = requests.get(url)

    # Verificar se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Extrair o conteúdo HTML da página
        html = response.text

        # Criar um objeto BeautifulSoup para analisar o HTML
        soup = BeautifulSoup(html, 'html.parser')

        # Encontrar todos os parágrafos (geralmente estão dentro de elementos <p>)
        all_paragraphs = soup.find_all('p')

        # Encontrar o capítulo do livro (geralmente estão dentro de elementos <h1>)
        chapter_book = soup.find_all('h1')
        chapter_book = chapter_book[0].get_text().strip()
        print(f"\ncapítulo: {chapter_book}\n")

        # Verificar cada parágrafo
        for i, paragraph in enumerate(all_paragraphs, start=1):
            if len(paragraph.get_text().split()) > 10:
              print(traducao(paragraph.get_text(), idioma))
              print("")

        print("-------------------------------------------------------------\n")

    else:
        print("Não foi possível acessar a página:", response.status_code)

Tradução de Capítulos

Digite para qual idioma deseja traduzir: 
inglês
capítulo: 5  ExpressÃµes multipalavras

The chapter on multiword expressions, which will be featured in the next edition of this book, will address a challenging topic in the field of Natural Language Processing (NLP). These expressions exist on the boundary between syntax and semantics, and always end up caught in the crossfire. On one hand, they present idiosyncrasies and specificities that do not allow for certain syntactic operations common to other sets of words. On the other hand, the definitions of the meaning of multiword expressions (or MWEs) generally deviate from the rule that the whole meaning is derived from the sum of its parts. Thankfully, because imagine if the expression "swallow the frog" were taken literally.

So, our intention for this chapter is, first and foremost, to define the fundamental concepts for those who will navigate the turbulent waters of the computational treatment of MWEs. Among 

####**CORREÇÃO GRAMATICAL DE TEXTO**
---



In [67]:
endpoint = "https://api.openai.com/v1/chat/completions"

def retornaParagrafoCorrigido(texto=0):
    if texto == 0:
      texto = ""

    mensagem_sistema = "Você é um analista gramatical que, ao ler um texto, retorna o mesmo texto só que corrigido com pontuações e palavras certas."
    mensagem_usuario = "O texto a ser analisado é: " + texto
    mensagem_assistente = "Retorne o mesmo texto, só que corrigido gramaticalmente. Evite frases como 'a correção fica: '. retorne diretamente apenas o texto corrigido"

    parameters = {
        "model": "gpt-3.5-turbo-0613",
        "max_tokens":1000,
        "temperature":0,
        "messages": [
            {"role": "system", "content": mensagem_sistema},
            {"role": "user", "content": mensagem_usuario},
            {"role": "assistant", "content": mensagem_assistente}
        ]
    }

    headers = {
        "Content-Type":"application/json",
        "Authorization": f"Bearer {chave_api}"
    }

    resposta = requests.post(endpoint, json=parameters, headers=headers)
    retorno_chat = resposta.json()

    novo_texto = retorno_chat.get('choices', [{}])[0].get('message', {}).get('content', 'error')
    if(novo_texto == 'error'):
        print(retorno_chat)

    return novo_texto

print("Correção gramatical de Capítulos\n")
for url in urls:

    # Fazer uma solicitação HTTP para obter o conteúdo da página
    response = requests.get(url)

    # Verificar se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Extrair o conteúdo HTML da página
        html = response.text

        # Criar um objeto BeautifulSoup para analisar o HTML
        soup = BeautifulSoup(html, 'html.parser')

        # Encontrar todos os parágrafos (geralmente estão dentro de elementos <p>)
        all_paragraphs = soup.find_all('p')

        # Encontrar o capítulo do livro (geralmente estão dentro de elementos <h1>)
        chapter_book = soup.find_all('h1')
        chapter_book = chapter_book[0].get_text().strip()
        print(f"capítulo: {chapter_book}\n")

        # Verificar cada parágrafo
        for i, paragraph in enumerate(all_paragraphs, start=1):

            #Analisa se o paragrafo tem mais de 10 palavras. Metodo para eliminar erros de buscas pela tag <p>
            if len(paragraph.get_text().split()) > 10:
              print(retornaParagrafoCorrigido(paragraph.get_text()))
              print("")

        print("-------------------------------------------------------------\n")

    else:
        print("Não foi possível acessar a página:", response.status_code)


Correção gramatical de Capítulos

capítulo: 5  ExpressÃµes multipalavras

O capítulo sobre as expressões multipalavras, que sairá na próxima edição deste livro, vai abordar um tema indigesto no universo do Processamento de Linguagem Natural (PLN). É que essas expressões ficam no limite entre a sintaxe e a semântica e sempre acabam ficando no meio do fogo cruzado. De um lado, elas apresentam idiossincrasias e especificidades que não permitem determinadas operações sintáticas comuns a outros conjuntos de palavras. De outro, as definições do sentido das expressões multipalavras (ou MWEs) em geral fogem à regra de que o significado do todo se dá pela soma das partes. Ainda bem, pois imagine se a expressão "engolir o sapo" fosse literal.

O texto corrigido é: Assim, a nossa intenção para este capítulo é, em primeiro lugar, definir quais são os conceitos fundamentais para quem vai navegar pelas águas turbulentas do tratamento computacional de MWEs. Entre esses conceitos, começaremos com a di